In [ ]:
import numpy as np, matplotlib.pyplot as plt
import pandas as pd, seaborn as sb

from anianssonwall.anianssonwall import *

# 1. Toy model

Demo kinetics for the simplified toy model used by Danov et al. (2006):

In [ ]:
n_max = 50
model = AnianssonWallCartoon(n_max)
n = model.n
ceq = model.steady_state(n)
phi = np.sum(n * ceq)

t_eval = [1e0, 1e1, 1e2]
c0 = np.zeros(n_max)
c0[0] = phi
traj = model.integrate(c0, t_eval[-1], t_eval=t_eval).T

# Show final decay mode as slowest eigenvector in steady-state:
J = model.jacobian_rhs(ceq)
evals, V = np.linalg.eig(J)
order = np.argsort(np.abs(evals))
evals, V = evals[order], V[:,order]
assert np.abs(evals[0]) < 1e-6 # steady-state mode

plt.figure(figsize=(3.375, 3))
V[:,0] *= np.sign(V[0,0])
V[:,0] *= np.sum(n*ceq) / np.sum(V[:,0])
V[:,1] *= np.sign(V[0,1])
assert np.allclose(V[:,0], n*ceq, rtol=1e-4, atol=1e-4)
plt.step(n, V[:,1], where='mid')
plt.xlabel('$n$')
plt.ylabel(r'$v_n^\infty$')

# Evolution of free monomers (not in true micelle, using 6 monomer as cutoff)
t_eval2 = np.geomspace(1e-2, 25, 1000)
traj2 = model.integrate(c0, t_eval2[-1], t_eval=t_eval2).T
traj2_free, traj2_micelle = traj2.copy(), traj2.copy()
traj2_free[:,model.n_local_maximum:] = 0
traj2_micelle[:,:model.n_local_maximum] = 0
c_free = traj2_free.dot(n)
c_micelle = traj2_micelle.dot(n)

plt.figure(figsize=(3.375, 3))
plt.plot(t_eval2, c_free/phi, label='free')
plt.plot(t_eval2, c_micelle/phi, label='micelle')
plt.xlabel('$t$')
plt.ylabel('$p$')
plt.legend(loc='best')

In [ ]:

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(3.375, 3.375),
                                height_ratios=[1, 1])

ax1.step(n, ceq, where='mid', label=r'$\infty$')
evals = {r'$\infty$': np.log10(-model.characteristic_rates(ceq))}

for c, tt in zip(reversed(traj), reversed(t_eval)):
    label = '$10^{:.0f}$'.format(np.log10(tt))
    evals[label] = np.log10(-model.characteristic_rates(c))
    pl, = ax1.step(n, c, where='mid', label=label)
    ax2.axvline(x=np.log10(1/tt), ls='--', c=pl.get_color())

evals = pd.DataFrame(evals)
sb.boxplot(ax=ax2, data=evals, showfliers=False, orient='h',
            width=0.95, linewidth=0.5, boxprops={'facecolor':'None'})
sb.swarmplot(ax=ax2, data=evals, orient='h',
            edgecolor=['k']*len(evals), linewidth=0.25, size=2.5)

ax1.legend(loc='best', title='$t$')
ax1.set_xlabel('$n$')
ax1.set_ylabel('$c_n(t)$')
ax1.set_xlim([0, n_max])
ax1.set_ylim([0, 1.65])

import matplotlib.ticker as ticker
ax2.xaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: rf'$10^{{{y:g}}}$'))
ax2.spines[['right', 'top', 'left']].set_visible(False)
ax2.set_yticks([])
ax2.set_xlabel(r'$|\lambda_i|$')

plt.show()

# 2. Maibaum model

Demo the more realistic thermodynamic model of Maibaum et al. (2004). This demo looks at the evolution of the full concentration profile and the relaxation modes.

First, set up the model:

In [ ]:
ratio = 0.2
free = 0.01
n_mode = 20
density = 0.01
g0 = 0.5

model = AnianssonWallMaibaum(ratio, free, n_mode, density, g0=g0)
n = model.n
n_max = model.n_max
ceq = model.steady_state(n)
phi = np.sum(n * ceq)
assert (phi - density)**2 < 1e-8

# Show final decay mode as slowest eigenvector in steady-state:
Jeq = model.jacobian_rhs(ceq)
evals, Veq = np.linalg.eig(Jeq)
order = np.argsort(np.abs(evals))
evals, Veq = evals[order], Veq[:,order]
assert np.abs(evals[0]) < 1e-6 # steady-state mode
Veq[:,0] *= np.sign(Veq[0,0])
Veq[:,0] *= np.sum(n*ceq) / np.sum(Veq[:,0])
Veq[:,1] *= np.sign(Veq[0,1])
Veq[:,2] *= -np.sign(Veq[0,2])
assert np.allclose(Veq[:,0], n*ceq, rtol=1e-4, atol=1e-4)

Now integrate the model. This may take some time (~15 minutes on my machine).

In [ ]:
N = 250
t_eval = np.geomspace(1e-2, 1e5, 1 + 7*N)

t_bifurcations = [25118, 71779] # rough estimate of lower bound
for t in t_bifurcations:
    select = np.argmin((t_eval - t)**2)
    t1, t2 = t_eval[select], t_eval[select+1]
    t_eval = np.sort(np.concatenate([t_eval, np.linspace(t1, t2, N)[1:-1]]))

assert 1e4 in t_eval
assert 1e5 in t_eval

# # Uncomment for a shorter test run
# t_eval = np.geomspace(1e-2, 1e3, 1 + 5*N)
# assert 1e1 in t_eval
# assert 1e2 in t_eval
# assert 1e3 in t_eval

c0 = np.zeros(n_max)
c0[0] = density
traj = model.integrate(c0, t_eval[-1], t_eval=t_eval).T

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, figsize=(3.375, 5))

ax1.step(n, ceq, where='mid', label=r'$\infty$')
evals = {r'$\infty$': np.log10(-model.characteristic_rates(ceq))}

t_show = [1e4, 1e5]
# t_show = [1e2, 1e3] # uncomment for shorter test runs
t_show = t_bifurcations

nsaddle = model.n_local_maximum
print(model.n_local_maximum, model.distribution.local_maximum, model.distribution.local_minimum)

c = ceq
C = np.sum(c)
n1 = np.sum(n*c) / C
assert np.isclose(n1*C, phi)
n2 = np.sum(n**2*c) / C
pfree = np.sum((n*c)[n < nsaddle]) / phi
psaddle = nsaddle * c[n == nsaddle][0] / phi
nmode = n[np.argmax(c)]
pleft = np.sum((n*c)[n < nmode]) / phi
pright = np.sum((n*c)[n > nmode]) / phi
nmicelle = np.sum((n*c)[n > nsaddle]) / np.sum(c[n > nsaddle])
print(f'nmode={nmode} <n>={n1:<6.4g} <nmic>={nmicelle:<6.4g} <n²>={n2:<6.4g} σ²={n2 - n1**2:<6.4g} σ={np.sqrt(n2 - n1**2):<6.4g} C={C:<6.4f} pfree={pfree:<6.3f} psaddle={psaddle:<6.3f} p(n<nmode)={pleft:<6.3f} p(n>nmode)={pright:<6.3f}')

for t in reversed(t_show):
    assert t in t_eval
    i = np.where(t_eval == t)[0]
    c = traj[i].reshape(-1)

    C = np.sum(c)
    n1 = np.sum(n*c) / C
    assert np.isclose(n1*C, phi)
    n2 = np.sum(n**2*c) / C
    pfree = np.sum((n*c)[n < nsaddle]) / phi
    psaddle = nsaddle * c[n == nsaddle][0] / phi
    nmode = n[np.argmax(c)]
    pleft = np.sum((n*c)[n < nmode]) / phi
    pright = np.sum((n*c)[n > nmode]) / phi
    nmicelle = np.sum((n*c)[n > nsaddle]) / np.sum(c[n > nsaddle])
    print(f'nmode={nmode} <n>={n1:<6.4g} <nmic>={nmicelle:<6.4g} <n²>={n2:<6.4g} σ²={n2 - n1**2:<6.4g} σ={np.sqrt(n2 - n1**2):<6.4g} C={C:<6.4f} pfree={pfree:<6.3f} psaddle={psaddle:<6.3f} p(n<nmode)={pleft:<6.3f} p(n>nmode)={pright:<6.3f}')

    exponent = np.floor(np.log10(abs(t)))
    leading = np.abs(t) / (10 ** exponent)
    label = f'10^{exponent:.0f}'
    if leading != 1.: label = fr'{leading:.1f} \times {label}'
    label = f'${label}$'
    evals[label] = np.log10(-model.characteristic_rates(c))
    pl, = ax1.step(n, c, where='mid', label=label)
    ax2.axvline(x=np.log10(1/t), ls='--', c=pl.get_color())

evals = pd.DataFrame(evals)
sb.boxplot(ax=ax2, data=evals, showfliers=False, orient='h',
            width=0.95, linewidth=0.5, boxprops={'facecolor':'None'})
sb.swarmplot(ax=ax2, data=evals, orient='h',
            edgecolor=['k']*len(evals), linewidth=0.25, size=2.5)

ax1.legend(loc='best', title='$t$')
ax1.set_xlabel('$n$')
ax1.set_ylabel('$c_n(t)$')
ax1.set_xlim([0, 40])
ax1.set_yscale('log')
ax1.set_ylim([1e-6, 1e-3])

import matplotlib.ticker as ticker
ax2.xaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: rf'$10^{{{y:g}}}$'))
ax2.spines[['right', 'top', 'left']].set_visible(False)
ax2.set_yticks([])
ax2.set_xlabel(r'$|\lambda_i|$')

pl, = ax3.step(n, Veq[:,1], where='mid')
ax3.step(n, Veq[:,2], where='mid', ls='dashed', c=pl.get_color())

for t in reversed(t_show):
    assert t in t_eval
    i = np.where(t_eval == t)[0]
    c = traj[i].reshape(-1)

    J = model.jacobian_rhs(c)
    evals, V = np.linalg.eig(J)
    order = np.argsort(np.abs(evals))
    evals, V = evals[order], V[:,order]
    V[:,1] *= np.sign(np.real(V[n==n_mode,1][0]))
    V[:,2] *= np.sign(np.real(V[n==n_mode,2][0]))
    assert np.abs(evals[0]) < 1e-6 # steady-state mode

    x = 1 - c[n == (nsaddle+1)][0] / c[n == nsaddle][0]
    y = np.real(1 - V[n == (nsaddle+1), 1][0] / V[n == nsaddle, 1][0])
    print(f'x={x:<6.4g} y={y:<6.4g}')

    pl, = ax3.step(n, np.real(V[:,1]), where='mid')
    ax3.step(n, np.real(V[:,2]), where='mid', ls='dashed', c=pl.get_color())
    # v = V[2] - 2 * V[1]
    # # v /= np.linalg.norm(v)
    # ax3.step(n, v, where='mid', ls='dotted', c=pl.get_color())

ax3.set_xlabel('$n$')
ax3.set_ylabel(r'$\mathrm{Re}(v_n^\text{slow})$')
# ax3.set_xlim([0, 40])
# ax3.set_ylim([-0.4, 0.2])

label = plt.text(0.975, 0., r'\textbf{a}', transform=ax1.transAxes, fontsize=18, ha='right', va='bottom')
label.set_in_layout(False)
label = plt.text(-0.15, 0.8, r'\textbf{b}', transform=ax2.transAxes, fontsize=18, ha='left', va='bottom')
label.set_in_layout(False)
label = plt.text(-0.15, 0.8, r'\textbf{c}', transform=ax3.transAxes, fontsize=18, ha='left', va='bottom')
label.set_in_layout(False)

plt.show()

In [ ]:
traj_free, traj_micelle = traj.copy(), traj.copy()
traj_free[:,model.n_local_maximum:] = 0
traj_micelle[:,:model.n_local_maximum] = 0
c_free = traj_free.dot(n)
c_micelle = np.sum(traj_micelle, axis=1)
n_micelle = np.sum(n*traj_micelle, axis=1) / c_micelle
n_total = np.sum(n*traj, axis=1) / np.sum(traj, axis=1)
n2_total = np.sum(n**2 * traj, axis=1) / np.sum(traj, axis=1)
n2_micelle = np.sum(n**2 * traj_micelle, axis=1) / c_micelle

fig, (ax1, ax3) = plt.subplots(nrows=2, figsize=(3.375, 3.375), sharex=True)

ax1.plot(t_eval, c_free)
pl, = ax1.plot(np.nan, np.nan)
ax2 = ax1.twinx()
ax2.plot(t_eval, c_micelle, c=pl.get_color())
ax4 = ax3.twinx()
ax3.plot(t_eval, n_total)
ax3.plot(t_eval, (n2_total - n_total**2), 'k--')
ax4.plot(t_eval, n_micelle, c=pl.get_color())
ax4.plot(t_eval, (n2_micelle - n_micelle**2), '--', c=pl.get_color())

index = np.argmax(c_micelle)
ax2.axvline(x=t_eval[index], ls='dashed', c=pl.get_color())

for ax in [ax1, ax3]: ax.set_xlabel('$t$')
ax1.set_ylabel('$C_\\mathrm{free}$')
ax2.set_ylabel('$C_\\mathrm{mic}$', c=pl.get_color())
ax2.tick_params(axis='y', labelcolor=pl.get_color())

for ax in [ax3, ax4]:
    ax.set_ylim([0, 9])
    ax.set_yticks(np.arange(1, 10, 2))
    ax.set_yticks(np.arange(0, 10, 2), minor=True)

ax3.set_ylabel(r'$\langle n \rangle$')
ax4.set_ylabel(r'$\langle n \rangle_\text{mic}$', c=pl.get_color())
ax4.tick_params(axis='y', labelcolor=pl.get_color())

plt.show()

Let's look at how the rate of the slowest eigenmode evolves in time ("aging"):

In [ ]:
nshow = 4
slowest_rates = np.empty((t_eval.size, nshow))

for i, c in enumerate(traj):
    J = model.jacobian_rhs(c)
    evals, V = np.linalg.eig(J)
    order = np.argsort(np.abs(evals))
    evals, V = evals[order], V[:,order]
    assert np.abs(evals[0]) < 1e-6 # steady-state mode

    slowest_rates[i] = np.abs(evals[1:1+nshow])
    # slowest_rates[i] = 1/np.imag(evals[1:1+nshow])

plt.figure(figsize=(3.375, 2.1))
plt.plot(t_eval, np.abs(slowest_rates), '.-')
plt.xlabel('$t$')
# plt.ylabel(r'$|\tau_i|$')
plt.ylabel('$|\lambda_i|$')
plt.xlim([0, t_eval[-1]])
plt.yscale('log')
plt.ylim([1e-6, 1e-4])

# t_bifurcation1 = 2.512e4
# t_bifurcation2 = 7.179e4
t_bifurcation1 = 2.5193e4
t_bifurcation2 = 7.2337e4

select = np.argmin((t_eval - t_bifurcation1)**2)
t_bifurcation1_bounds = t_eval[select], t_eval[select+1]
# print(t_bifurcation1_bounds)
t_bifurcation1 = t_eval[select]

select = np.argmin((t_eval - t_bifurcation2)**2)
t_bifurcation2_bounds = t_eval[select], t_eval[select+1]
# print(t_bifurcation2_bounds)
t_bifurcation2 = t_eval[select]

t_bifurcations = [t_bifurcation1, t_bifurcation2]
for t in t_bifurcations: plt.axvline(x=t, ls='dashed')

# plt.xlim([7.15e4, 7.3e4])
# plt.ylim([3.12e-6, 3.27e-6])
# plt.xlim([7.233e4, 7.234e4])
# plt.ylim([3.195e-6, 3.205e-6])

# plt.xlim([2.51e4, 2.54e4])
# plt.ylim([7.3e-6, 7.8e-6])
plt.xlim([2.519e4, 2.5195e4])
plt.ylim([7.55e-6, 7.59e-6])

plt.show()